<a href="https://colab.research.google.com/github/Rafe2001/Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('/content/output_chatbot.json') as file:
  data = json.load(file)

In [3]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])

  if intent['tag'] not in labels:
    labels.append(intent['tag'])
num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 50
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen = max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
1/1 [==============================] - 6s 6s/step - loss: 2.9443 - accuracy: 0.0526
Epoch 2/500
1/1 [==============================] - 0s 12ms/step - loss: 2.9441 - accuracy: 0.0526
Epoch 3/500
1/1 [==============================] - 0s 17ms/step - loss: 2.9440 - accuracy: 0.1053
Epoch 4/500
1/1 [==============================] - 0s 14ms/step - loss: 2.9439 - accuracy: 0.1053
Epoch 5/500
1/1 [==============================] - 0s 12ms/step - loss: 2.9437 - accuracy: 0.1053
Epoch 6/500
1/1 [==============================] - 0s 13ms/step - loss: 2.9436 - accuracy: 0.1053
Epoch 7/500
1/1 [==============================] - 0s 13ms/step - loss: 2.9434 - accuracy: 0.1053
Epoch 8/500
1/1 [==============================] - 0s 14ms/step - loss: 2.9433 - accuracy: 0.1579
Epoch 9/500
1/1 [==============================] - 0s 16ms/step - loss: 2.9431 - accuracy: 0.1579
Epoch 10/500
1/1 [==============================] - 0s 14ms/step - loss: 2.9430 - accuracy: 0.2105
Epoch 11/500
1/1 [===

In [8]:
model.save("chat_model.h5")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)